## Preperation: load dependencies

In [6]:
interp.load.ivy("com.typesafe.akka" %% "akka-http-spray-json" % "10.2.4")
interp.load.ivy("com.typesafe.slick" %% "slick" % "3.3.3")
interp.load.ivy("com.typesafe.slick" %% "slick-hikaricp" % "3.3.3")
interp.load.ivy("com.github.tminglei" %% "slick-pg" % "0.19.5")
interp.load.ivy("com.github.tminglei" %% "slick-pg_spray-json" % "0.19.5")
interp.load.ivy("org.postgresql" % "postgresql" % "42.2.19")
interp.load.ivy("ch.qos.logback" % "logback-classic" % "1.2.3")

// Load .jar
interp.load.cp(os.Path("/home/jovyan/jar/masterthesis-khipu_2.13-0.1.jar"))

In [16]:
// Smile

import $ivy.`com.github.haifengl::smile-scala:2.6.0`

import scala.language.postfixOps
import org.apache.commons.csv.CSVFormat
import smile._
import smile.util._
import smile.math._
import smile.math.MathEx._
import smile.math.kernel._
import smile.math.matrix._
import smile.math.matrix.Matrix._
import smile.math.rbf._
import smile.stat.distribution._
import smile.data._
import smile.data.formula._
import smile.data.measure._
import smile.data.`type`._
import smile.json._
import smile.interpolation._
import smile.validation._
import smile.association._
import smile.base.cart.SplitRule
import smile.base.mlp._
import smile.base.rbf.RBF
import smile.classification._
import smile.regression._
import smile.feature._
import smile.clustering._
import smile.vq._
import smile.manifold._
import smile.mds._
import smile.sequence._
import smile.projection._
import smile.nlp._
import smile.wavelet._
// Plotting
import smile.plot.swing._
import smile.plot.vega._
import smile.plot.show
import smile.plot.Render._

import $ivy.$                                       


import scala.language.postfixOps

import org.apache.commons.csv.CSVFormat

import smile._

import smile.util._

import smile.math._

import smile.math.MathEx._

import smile.math.kernel._

import smile.math.matrix._

import smile.math.matrix.Matrix._

import smile.math.rbf._

import smile.stat.distribution._

import smile.data._

import smile.data.formula._

import smile.data.measure._

import smile.data.`type`._

import smile.json._

import smile.interpolation._

import smile.validation._

import smile.association._

import smile.base.cart.SplitRule

import smile.base.mlp._

import smile.base.rbf.RBF

import smile.classification._

import smile.regression._

import smile.feature._

import smile.clustering._

import smile.vq._

import smile.manifold._

import smile.mds._

import smile.sequence._

import smile.projection._

import smile.nlp._

import smile.wavelet._
// Plotting

import smile.plot.swing._

import smile.plot.vega._

i

## Get data from DB

In [17]:
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.Future
import scala.concurrent.duration._
import scala.concurrent.{ Await, Future }
import scala.language.postfixOps

import de.tmrdlt.database.workflowlist.WorkflowListDB
import de.tmrdlt.database.action.ActionDB


val workflowListDB = new WorkflowListDB()
val actionDB = new ActionDB()

val workflowLists =  Await.result(workflowListDB.getWorkflowLists, 2 seconds)
val actions =  Await.result(actionDB.getActions, 2 seconds)
val moveActions =  Await.result(actionDB.getMoveActions, 2 seconds)

import scala.concurrent.ExecutionContext.Implicits.global

import scala.concurrent.Future

import scala.concurrent.duration._

import scala.concurrent.{ Await, Future }

import scala.language.postfixOps


import de.tmrdlt.database.workflowlist.WorkflowListDB

import de.tmrdlt.database.action.ActionDB



workflowListDB: WorkflowListDB = de.tmrdlt.database.workflowlist.WorkflowListDB@39a2815e
actionDB: ActionDB = de.tmrdlt.database.action.ActionDB@5e3c2c60
workflowLists: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
...
actions: Seq[de.tmrdlt.database.action.Action] = Vector(
  Action(
...
moveActions: Seq[de.tmrdlt.database.moveaction.MoveAction] = Vector(
  MoveAction(1L, 35L, "5fad0555e165558a64ad0479", "5fad0555e165558a64ad047a"),
...

## Size of Board over time

In [22]:
import de.tmrdlt.models._
import java.time.LocalDate
import scala.collection.mutable.ListBuffer

val boards = workflowLists.filter(_.listType == WorkflowListType.BOARD)
val board = boards.head

val data = new ListBuffer[(Int, LocalDate)]
data += ((0, board.createdAt.toLocalDate))

val sortedActions = actions
    .filter(a => a.actionType == "createCard" || a.actionType == "deleteCard")
    .sortBy(_.date)

sortedActions.foreach{a => 
    val count = if (a.actionType == "createCard") data.last._1 + 1 else data.last._1 - 1
    data += ((count, a.date.toLocalDate))}


import de.tmrdlt.models._

import java.time.LocalDate

import scala.collection.mutable.ListBuffer


boards: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
...
board: de.tmrdlt.database.workflowlist.WorkflowList = WorkflowList(
  1L,
...
data: ListBuffer[(Int, LocalDate)] = ListBuffer(
  (0, 2020-11-12),
...
res21_6: ListBuffer[(Int, LocalDate)] = ListBuffer(
  (0, 2020-11-12),
...
sortedActions: Seq[de.tmrdlt.database.action.Action] = Vector(
  Action(
...

In [25]:
val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
val bar = VegaLite.view().
  mark("bar").
  heightStep(40).
  x(field = "date", `type` = "temporal").
  y(field = "count", `type` = "quantitative").
  data(smileData).
  description("Size of the boards over time")

show(bar)

<iframe id="d6970f8c-f9d2-43e5-aa62-8bc9bb2da7b4" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-12","count":0},{"date":"2020-11-12","count":1},{"date":"2020-11-12","count":2},{"date":"2020-11-12","count":3},{"date":"2020-11-12","count":4},{"date":"2020-11-30","count":5},{"date":"2020-11-30","count":6},{"date":"2020-11-30","count":7},{"date":"2020-11-30","count":8},{"date":"2020-11-30","count":9},{"date":"2020-12-01","count":10},{"date":"2020-12-01","count":11},{"date":"2020-12-01","count":12},{"date":"2020-12-01","count":13},{"date":"2020-12-04","count":14},{"date":"2020-12-04","count":15},{"date":"2020-12-07","count":14},{"date":"2020-12-07","count":13},{"date":"2020-12-07","count":14},{"date":"2020-12-08","count":15},{"date":"2020-12-14","count":16},{"date":"2021-01-25","count":17},{"date":"2021-02-02","count":18},{"date":"2021-02-02","count":19},{"date":"2021-02-02","count":20},{"date":"2021-02-09","count":21},{"date":"2021-02-15","count":22},{"date":"2021-02-15","count":23},{"date":"2021-02-22","count":24},{"date":"2021-02-23","count":25},{"date":"2021-02-26","count":26},{"date":"2021-03-08","count":27},{"date":"2021-03-08","count":28}]},"description":"Size of the boards over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":"bar","height":{"step":40}};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

smileData: List[JsObject] = List(
  JsObject(HashMap("date" -> JsDate(2020-11-12), "count" -> JsInt(0))),
...
bar: VegaLite = {
  "data": {
...

## Size of columns over time

In [26]:
val columns = workflowLists
    .filter(_.listType == WorkflowListType.LIST)
    .filter(_.parentId == Some(board.id))

val relevantActions = actions.filter(a => a.actionType == "createCard" ||
                                     a.actionType == "deleteCard" ||
                                     moveActions.filter(_.actionId == a.id).size > 0)

columns.foreach { column =>
    val data = new ListBuffer[(Int, LocalDateTime)]

    
    
    
    
}


columns: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
...
relevantActions: Seq[de.tmrdlt.database.action.Action] = Vector(
  Action(
...